### Train-test Split

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# train-test split

x = train
y = train['target']
del train['target']    #or train.remove('target')

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=1,stratify=y)

print ("The train data has", train.shape)
print ("The test data has", test.shape)

### Feature Scaling

In [ ]:
x_train.drop(['Id'], axis=1, inplace=True)
x_test.drop(['Id'], axis=1, inplace=True)
scaler = MinMaxScaler()

scaler.fit(x_train)
x_train_new = scaler.transform(x_train)

scaler.fit(x_test)
x_test_new = scaler.transform(x_test)

### Cross-validated Score

In [ ]:
baseline = cross_val_score(ElasticNet(random_state=123), x_train, y_train, 
                           scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)

print('Baseline MAE from using ElasticNet is: {.3f}'.format(baseline.mean()))

In [ ]:
baseline = cross_validate(RandomForestRegressor(n_estimators=100, random_state=123), x_train, y_train, 
                          scoring='neg_mean_absolute_error', cv=5, n_jobs=-1, error_score='raise')

baseline['test_score'].mean()

### LightGBM

In [ ]:
# LGB as a potentially improved model

lgb = LGBRegressor(random_state=123)
lgb.fit(x_train, y_train, eval_set=[(x_val,y_val)], eval_metric='mae', early_stopping_rounds=30, verbose=20)
y_pred = lgb.predict(x_val)

print('Improved MAE from using LightGBM Regressor is: {.3f}'.format(mean_absolute_error(y_val, y_pred)))

# or we can use the cv method

dftrainLGB = lgb.Dataset(data = x_train, label = y_train, feature_name = x_train.columns.tolist())
params = {'objective': 'regression'}
lgb.cv(
    params,
    dftrainLGB,
    num_boost_round=100,
    nfold=3,
    metrics='mae',
    early_stopping_rounds=10
)